In [9]:
import pathlib
import numpy as np
import keras
import tensorflow
from keras.models import Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Input, Add, ReLU, AveragePooling2D, GlobalAveragePooling2D, ZeroPadding2D

path = 'C:\\Users\\pc\\Desktop\\GP\\256_ObjectCategories\\256_ObjectCategories\\'
img_size = (64, 64)
input_shape = (64, 64, 3)
num_classes = 257

In [10]:
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

In [13]:
dataDirectory = pathlib.Path(path)

train_data_set = tensorflow.keras.preprocessing.image_dataset_from_directory(
    dataDirectory,
    labels = 'inferred',
    label_mode = 'categorical',
    validation_split = 0.2,
    subset = 'training',
    seed = 1234,
    image_size = img_size,
    batch_size = 128
)
train_data_set = train_data_set.map(process)

val_data_set = tensorflow.keras.preprocessing.image_dataset_from_directory(
    dataDirectory,
    labels = 'inferred',
    label_mode = 'categorical',
    validation_split = 0.2,
    subset = 'validation',
    seed = 1234,
    image_size = img_size,
    batch_size = 128
)

val_data_set = val_data_set.map(process)
AUTOTUNE = tensorflow.data.experimental.AUTOTUNE
train_data_set = train_data_set.cache().prefetch(buffer_size = AUTOTUNE)
val_data_set = val_data_set.cache().prefetch(buffer_size = AUTOTUNE)

Found 30607 files belonging to 257 classes.
Using 24486 files for training.
Found 30607 files belonging to 257 classes.
Using 6121 files for validation.


In [56]:
def identity_block(X, f, filters, stage, block) :
    
    F1, F2, F3 = filters
    X_shortCut = X
    
    X = Conv2D(filters = F1,
              kernel_size = 1,
              strides = 1,
              padding = 'valid',
              kernel_initializer = keras.initializers.glorot_uniform(seed = 0)
              )(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F2,
              kernel_size = f,
              strides = 1,
              padding = 'same',
              kernel_initializer = keras.initializers.glorot_uniform(seed = 0)
              )(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F3,
              kernel_size = 1,
              strides = 1,
              padding = 'valid',
              kernel_initializer = keras.initializers.glorot_uniform(seed = 0)
              )(X)
    X = BatchNormalization(axis = 3)(X)
    
    X = Add()([X, X_shortCut])
    X = Activation('relu')(X)
    
    return X

In [57]:
def convolutional_block(X, f, filters, stage, block, s = 2) :
    
    F1, F2, F3 = filters
    X_shortCut = X
    
    X = Conv2D(filters = F1,
              kernel_size = 1,
              strides = s,
              padding = 'valid',
              kernel_initializer = keras.initializers.glorot_uniform(seed = 0)
              )(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F2,
              kernel_size = f,
              strides = 1,
              padding = 'same',
              kernel_initializer = keras.initializers.glorot_uniform(seed = 0)
              )(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F3,
              kernel_size = 1,
              strides = 1,
              padding = 'valid',
              kernel_initializer = keras.initializers.glorot_uniform(seed = 0)
              )(X)
    X = BatchNormalization(axis = 3)(X)
    
    X_shortCut = Conv2D(filters = F3,
                       kernel_size = 1,
                       strides = s,
                       padding = 'valid',
                       kernel_initializer = keras.initializers.glorot_uniform(seed = 0)
                       )(X_shortCut)
    X_shortCut = BatchNormalization(axis = 3)(X_shortCut)
    
    X = Add()([X, X_shortCut])
    X = Activation('relu')(X)
    
    return X

In [61]:
def ResNet(input_shape_ = input_shape, classes = num_classes) :
    X_input = Input(input_shape_)
    
    X = ZeroPadding2D((3,3))(X_input)
    
    X = Conv2D(filters = 64,
              kernel_size = 7,
              strides = 2,
              name = 'conv1',
              kernel_initializer = keras.initializers.glorot_uniform(seed = 0)
              )(X)
    X = BatchNormalization(axis = 3, name = 'BN_Conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides = (2,2))(X)
    
    X = convolutional_block(X, f = 3, filters = [64,64,256], stage =  2, block = 'a', s = 1)
    X = identity_block(X, 3, [64,64,256], stage = 2, block = 'b')
    X = identity_block(X, 3, [64,64,256], stage = 2, block = 'c')
    
    X = convolutional_block(X, f = 3, filters = [128,128,512], stage =  3, block = 'a', s = 2)
    X = identity_block(X, 3, [128,128,512], stage = 3, block = 'b')
    X = identity_block(X, 3, [128,128,512], stage = 3, block = 'c')
    X = identity_block(X, 3, [128,128,512], stage = 3, block = 'd')
    
    X = convolutional_block(X, f = 3, filters = [256,256,1024], stage =  4, block = 'a', s = 2)
    X = identity_block(X, 3, [256,256,1024], stage = 4, block = 'b')
    X = identity_block(X, 3, [256,256,1024], stage = 4, block = 'c')
    X = identity_block(X, 3, [256,256,1024], stage = 4, block = 'd')
    X = identity_block(X, 3, [256,256,1024], stage = 4, block = 'e')
    X = identity_block(X, 3, [256,256,1024], stage = 4, block = 'f')
    
    X = convolutional_block(X, f = 3, filters = [512,512,2048], stage = 5, block = 'a', s = 2)
    X = identity_block(X, 3, [512,512,2048], stage = 5, block = 'b')
    X = identity_block(X, 3, [512,512,2048], stage = 5, block = 'c')
    
    X = AveragePooling2D()(X)
    X = Flatten()(X)
    X = Dense(classes, activation = 'softmax', 
              kernel_initializer = keras.initializers.glorot_uniform(seed = 0))(X)
    
    model = Model(inputs = X_input, outputs = X, name = 'GP')
    return model

In [62]:
model = ResNet(input_shape_ = input_shape, classes = num_classes)
model.summary()

Model: "GP"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_14 (ZeroPadding2 (None, 70, 70, 3)    0           input_16[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_14[0][0]          
__________________________________________________________________________________________________
BN_Conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
_________________________________________________________________________________________________

In [7]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

In [8]:
model.fit(train_data_set, validation_data = val_data_set, epochs = 20)

Epoch 1/20
  3/192 [..............................] - ETA: 24:39 - loss: 7.5533 - accuracy: 0.0148     

KeyboardInterrupt: 